In [1]:
import logging
import numpy as np
import tensorflow as tf
from typing import Dict, List, Tuple
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import History
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import SGD

In [2]:
class NeuralNetwork:
    """
    Class for deploying neural network
    """
        
    def model_deploy(self, merge_models: tf.Tensor, inputs: List[tf.Tensor]) -> Model:
        """
        Creates a multi-input and multi-output neural network model 

        Args:
            merge_models: A tensor that combined continuous features and embeddings from categorical features
            inputs: A list containing the Input layer for each feature

        Returns:
            Model: A prepared structure of model 
        """
        try:
            
            x = Dense(300, activation='relu', kernel_regularizer=l2(0.01))(merge_models)
            x = BatchNormalization()(x)
            x = Dense(300, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(x)
            x = BatchNormalization()(x)

            home_score_output = Dense(1, activation=None, name="home_score")(x)
            away_score_output = Dense(1, activation=None, name="away_score")(x)

            model = Model(inputs=inputs, outputs=[home_score_output, away_score_output])
            optimizer = SGD(learning_rate=0.01)
            model.compile(optimizer=optimizer, 
                          loss={"home_score": "mae", "away_score": "mae"},
                          metrics={"home_score": RootMeanSquaredError(),
                                   "away_score": RootMeanSquaredError()                                  
                                   })
            model.summary()
            logging.info("Successfully created multi-input and multi-output neural network model")
            return model
        except Exception as e:
            logging.error(f"Error in the model deploying")
            raise e


    def model_train(self, 
                    model: Model, 
                    training_dataset: Dict[str, Dict[str, np.ndarray]], 
                    validation_dataset: Dict[str, Dict[str, np.ndarray]], 
                    epochs: int = 10
                    ) -> Tuple[Model, History]:
        """
        Trains the model on the training dataset and validate on new dataset

        Args:
            model: Allready prepared model
            training_dataset: Training dataset
            validation_dataset: Validation dataset
            epochs: Number of epochs for model to train
        Returns:
            Model: Trained model
            History: History of metrics all epochs during training
        """
        try:
            logging.info("\nModel training:")
            
            training_features = training_dataset["input_features"]
            training_targets = training_dataset["targets"]
            validation_features = validation_dataset["input_features"]
            validation_targets = validation_dataset["targets"]
            
            history = model.fit(training_features, 
                      training_targets, 
                      epochs=epochs,
                      validation_data=(validation_features, 
                                       validation_targets)
                                )
            logging.info("Successfully trained the model")
            return model, history
        except Exception as e:
            logging.error(f"Error in model training: {e}")
            raise e